# **Callbacks**
- 학습 도중에 이벤트 발생 시키는 기능
- 체크포이트 중간 저장가능, 중간 마다 데이터 조작이나 진행 과정 알수 있다.

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

# **Hyperparameter Tunning**

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

# **Build Model**

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# **Data Preprocess**

In [5]:
os.listdir('D:/dataset/cifar/')

['labels.txt', 'test', 'train']

In [6]:
train_paths = glob('D:/dataset/cifar/train/*.png')[:1000]
test_paths = glob('D:/dataset/cifar/test/*.png')[:1000]

In [7]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [8]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [9]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [10]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [11]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [14]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# **Callbacks**

In [15]:
starttime = datetime.now()

In [16]:
starttime

datetime.datetime(2020, 11, 25, 12, 8, 49, 992482)

In [17]:
end_time = datetime.now()

In [18]:
end_time

datetime.datetime(2020, 11, 25, 12, 8, 50, 55488)

In [19]:
end_time - starttime

datetime.timedelta(microseconds=63006)

In [20]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [21]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir, 
    write_graph=True, 
    write_images=True,
    histogram_freq=1
)

https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/image_summaries.ipynb#scrollTo=IJNpyVyxbVtT

In [22]:
%tensorboard --logdir logs --port 8008

Reusing TensorBoard on port 8008 (pid 19008), started 20:55:29 ago. (Use '!kill 19008' to kill it.)

## **Training**
http://localgost:6006

In [23]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
 1/31 [..............................] - ETA: 0s - loss: 2.3446 - accuracy: 0.0312WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
31/31 [==============================] - 18s 591ms/step - loss: 2.3102 - accuracy: 0.1095 - val_loss: 2.2867 - val_accuracy: 0.1129
Epoch 2/10
31/31 [==============================] - 9s 283ms/step - loss: 2.2720 - accuracy: 0.1405 - val_loss: 2.2260 - val_accuracy: 0.1240
Epoch 3/10
31/31 [==============================] - 9s 281ms/step - loss: 2.1948 - accuracy: 0.1715 - val_loss: 2.1062 - val_accuracy: 0.2046
Epoch 4/10
31/31 [==============================] - 9s 303ms/step - loss: 2.1535 - accuracy: 0.1942 - val_loss: 2.0605 - val_accuracy

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup

In [24]:
# https://www.tensorflow.org/tensorboard/r2/image_summaries

import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [25]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [26]:
# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [27]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

Epoch 1/10
31/31 [==============================] - ETA: 0s - loss: 1.8253 - accuracy: 0.3120

<ipython-input-24-7b8efe693ea4>:43: RuntimeWarning: invalid value encountered in true_divide
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)


31/31 [==============================] - 14s 459ms/step - loss: 1.8253 - accuracy: 0.3120 - val_loss: 1.7949 - val_accuracy: 0.3276
Epoch 2/10
31/31 [==============================] - 11s 357ms/step - loss: 1.8063 - accuracy: 0.3285 - val_loss: 1.7728 - val_accuracy: 0.3448
Epoch 3/10
31/31 [==============================] - 11s 350ms/step - loss: 1.7128 - accuracy: 0.3698 - val_loss: 1.8211 - val_accuracy: 0.3427
Epoch 4/10
31/31 [==============================] - 11s 359ms/step - loss: 1.7627 - accuracy: 0.3409 - val_loss: 1.7065 - val_accuracy: 0.3649
Epoch 5/10
31/31 [==============================] - 11s 362ms/step - loss: 1.6719 - accuracy: 0.3729 - val_loss: 1.7492 - val_accuracy: 0.3649
Epoch 6/10
31/31 [==============================] - 12s 382ms/step - loss: 1.6747 - accuracy: 0.3729 - val_loss: 1.7235 - val_accuracy: 0.3720
Epoch 7/10
31/31 [==============================] - 14s 445ms/step - loss: 1.6713 - accuracy: 0.3915 - val_loss: 1.7720 - val_accuracy: 0.3659
Epoch 8/10

# **Expert**

In [28]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [29]:
train_loss = tf.keras.metrics.Mean(name = "train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuray')

In [43]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions) # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables) # 모델의 trainable_varialbe을 하여금 loss를 통해 기울기를 없음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # 구한 최적화된 값을 variable에 적용
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [31]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [46]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))

In [49]:
file_write = tf.summary.create_file_writer(logdir)

In [ ]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):
        train_step(images, labels)
        
    with file_write.as_default():
        tf.summary.Image('input_Image', images, step=step + (step * epoch))
        tf.summary.scalar('train_loss', train_loss.result(), step + (step * epoch))
        tf.summary.scalar('train_accuracy', train_accuracy.result(), step + (step * epoch))
    
    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)
        
    template = 'Epoch: {}, Loss: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result().
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result() *100))